<a href="https://colab.research.google.com/github/HandsomeJames0223/Deep_Learning/blob/main/sentiment(109_10_27).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 4s 0us/step


In [ ]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [ ]:
import glob
# glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd

def read(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    return content

def get_data(t):
    dn = os.path.dirname(dataset)
    pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
    pos = glob.glob(pattern)
    pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
    neg = glob.glob(pattern)
    sentiments = [1] * len(pos) + [0] * len(neg)
    contents = map(read, pos + neg)
    df = pd.DataFrame({
            "contents":contents,
            "sentiment":sentiments
        })
    return df

train_df = get_data("train")
test_df = get_data("test")
test_df

,contents,sentiment
0,"It kept me on the edge of my seat. True, the s...",1
1,Born in 1946 I was about eight years old when ...,1
2,In the 60's Cleveland television audiences cou...,1
3,I think it is a brilliant show with cool talki...,1
4,John Wayne's first starring role just blew me ...,1
...,...,...
24995,I am not a big fan of the Spielberg/Cruise ver...,0
24996,The director infuses this film with false dept...,0
24997,"As a flying and war movie buff, this ranks at ...",0
24998,I rented this movie under the impression that ...,0


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit_on_texts: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [ ]:
# tok.word_index
tok.index_word[2]

'and'

In [ ]:
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,1,17,6,84,2,10,37,1,62.0,10.0,2781.0,11.0,17.0,71.0,82.0,17.0,138.0,1.0,2753.0,1189.0,17.0,2.0,705.0,17.0,10.0,40.0,373.0,1.0,879.0,10.0,37.0,3.0,632.0,2.0,79.0,3.0,280.0,62.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,6,4,4,84,399,209,105,11.0,215.0,3.0,2103.0,4.0,2537.0,1.0,2782.0,1.0,290.0,106.0,6.0,2186.0,16.0,1474.0,844.0,2.0,87.0,70.0,15.0,3.0,129.0,4.0,24.0,62.0,41.0,86.0,24.0,993.0,938.0,24.0,444.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,3,193,55,764,334,10,40,66.0,5.0,137.0,64.0,1.0,17.0,9.0,13.0,84.0,134.0,47.0,67.0,112.0,27.0,192.0,409.0,926.0,36.0,1.0,764.0,808.0,47.0,68.0,84.0,660.0,4.0,46.0,4.0,58.0,511.0,764.0,1838.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,17,69,35,73,12,10,103,9.0,30.0,219.0,277.0,3.0,288.0,30.0,208.0,10.0,166.0,9.0,30.0,208.0,10.0,166.0,9.0,2.0,10.0,207.0,166.0,1.0,102.0,403.0,2.0,144.0,9.0,6.0,3.0,17.0,12.0,284.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1250,4,1976,1477,343,62,646,36.0,3.0,13.0,83.0,614.0,20.0,695.0,696.0,8.0,20.0,1.0,114.0,293.0,1305.0,2974.0,467.0,10.0,216.0,12.0,9.0,13.0,109.0,2444.0,171.0,20.0,2974.0,467.0,2.0,869.0,5.0,199.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,9,6,1295,12,3,226,14,1593,2.0,337.0,1753.0,97.0,94.0,98.0,265.0,57.0,1.0,389.0,28.0,1.0,411.0,8.0,11.0,17.0,163.0,303.0,37.0,3.0,309.0,1177.0,353.0,1511.0,295.0,1.0,75.0,2001.0,392.0,1643.0,1734.0,1298.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,3,19,37,477,1695,2,6,3,401.0,459.0,4.0,86.0,1.0,314.0,8.0,11.0,701.0,40.0,149.0,76.0,9.0,33.0,112.0,119.0,2.0,1992.0,5.0,1034.0,2.0,1034.0,80.0,989.0,7.0,7.0,11.0,19.0,2026.0,16.0,3.0,114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,6,1,1124,4,592,12,10,25.0,107.0,8.0,3.0,193.0,55.0,1.0,233.0,55.0,10.0,1796.0,3.0,17.0,35.0,73.0,13.0,1861.0,146.0,501.0,300.0,2.0,47.0,6.0,424.0,54.0,210.0,5.0,11.0,17.0,546.0,5.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,10,1081,1884,12,143,1644,1,1106,2.0,920.0,42.0,3.0,2265.0,2322.0,2660.0,135.0,13.0,10.0,2765.0,119.0,10.0,566.0,58.0,416.0,39.0,2503.0,29.0,4.0,58.0,97.0,9.0,27.0,1155.0,7.0,7.0,54.0,10.0,20.0,704.0,134.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# pd.DataFrame(train_df_seq)
# truncating: pre 12345 -> 45 post 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, 512)
x_test_pad = pad_sequences(x_test_seq, 512)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,3,1161,12,1,17,6,21,1867,14,45,12,68,3,2178,15,49,19,280,54,39,1029,8,11,28,40,3,706,487,178,5,121,86,11,62,627,277,22,377,146
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,31,666,2041,1532,1,1437,606,37,3,247,8,58,702,9,13,37,316,32,776,877,9,90,3,524,133,12,73,430,276,54,28,77,166,11,730,18,9,1674,69
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,728,196,1,2,1,1393,1,106,13,1,29,55,115,265,1016,431,53,5,12,55,2,125,71,256,234,1558,6,305,7,7,92,171,48,78,10,121,143,61,3,504
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,792,23,134,22,200,64,86,9,492,8,1,1844,162,10,101,11,6,28,4,1,115,245,284,2,10,481,383,11,5,29,346,34,37,369,7,7,15,58,1911
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,97,1,254,248,6,32,1,995,1489,1847,23,2437,18,1,19,14,3,223,6,21,10,188,101,4,157,422,12,67,1556,1,191,8,3,278,4,578,476,20,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,822,12,230,2315,50,71,3,644,1755,8,1,624,485,307,35,5,10,479,10,25,5,137,16,1,30,219,9,405,1722,99,3,321,30,1,1710,1064,1214,1688,2335
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,169,2752,2,26,57,8,108,47,23,197,131,104,99,1,290,1467,109,12,109,47,1098,364,2,149,20,1464,2,5,984,3,582,134,995,180,72,190,15,2484,8,110
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,12,1,84,81,33,23,266,5,27,162,119,11,91,217,14,45,33,801,5,898,8,14,108,2869,14,611,47,23,108,49,1544,105,2,11,6,21,28,4,95
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,299,6,48,163,11,17,287,2739,7,7,1,315,941,1468,3,1234,646,80,1,174,2,1442,1919,4,1419,7,7,511,344,1,4,58,2037,177,5,11,52,558,4,55


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
  # 1~3000(token) + 0(padding)
  # params: 3001 * 128
  Embedding(3001, 128, mask_zero=True, input_length=512),
  GlobalAveragePooling1D(),
  Dense(2, activation="softmax")  
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [ ]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [ ]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
225/225 - 4s - loss: 0.5731 - accuracy: 0.7124 - val_loss: 0.5108 - val_accuracy: 0.7632
Epoch 2/50
225/225 - 3s - loss: 0.3602 - accuracy: 0.8630 - val_loss: 0.3710 - val_accuracy: 0.8448
Epoch 3/50
225/225 - 4s - loss: 0.2907 - accuracy: 0.8877 - val_loss: 0.3575 - val_accuracy: 0.8484
Epoch 4/50
225/225 - 3s - loss: 0.2594 - accuracy: 0.9008 - val_loss: 0.3642 - val_accuracy: 0.8460
Epoch 5/50
225/225 - 4s - loss: 0.2405 - accuracy: 0.9080 - val_loss: 0.3109 - val_accuracy: 0.8732
Epoch 6/50
225/225 - 3s - loss: 0.2279 - accuracy: 0.9122 - val_loss: 0.3423 - val_accuracy: 0.8592
Epoch 7/50
225/225 - 4s - loss: 0.2187 - accuracy: 0.9163 - val_loss: 0.3433 - val_accuracy: 0.8612
Epoch 8/50
225/225 - 3s - loss: 0.2116 - accuracy: 0.9197 - val_loss: 0.3478 - val_accuracy: 0.8596
Epoch 9/50
225/225 - 3s - loss: 0.2065 - accuracy: 0.9222 - val_loss: 0.3198 - val_accuracy: 0.8736
Epoch 10/50
225/225 - 3s - loss: 0.2019 - accuracy: 0.9239 - val_loss: 0.3490 - val_accuracy: 0.8620

In [ ]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 3ms/step - loss: 0.2863 - accuracy: 0.8836


[0.28634870052337646, 0.8835600018501282]

In [ ]:
layers = [
  Embedding(3001, 128, mask_zero=True),
  GlobalAveragePooling1D()
]
w = model.layers[0].get_weights()
infer = Sequential(layers)
infer.layers[0].set_weights(w)
infer.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# w
# infer.layers[0].get_weights()
# 25000, 512
# [512list, 512list]
target = "the"
# [1list]
pre = infer.predict([[tok.word_index[target]]])
pre[0]

array([-0.02148957, -0.03330651, -0.01124272,  0.03045076, -0.00856636,
        0.04757436,  0.00769672, -0.01014575, -0.05740415,  0.01701728,
        0.00246252, -0.00923929, -0.01776502, -0.00385821,  0.00841107,
        0.04086614, -0.01950323,  0.03012832, -0.05279728,  0.02719372,
       -0.00786472, -0.02832714, -0.01920153, -0.01598269, -0.00875571,
       -0.03387471, -0.03219924, -0.03252126, -0.0036557 , -0.0215464 ,
       -0.05654101,  0.00891035, -0.0499809 ,  0.04139885, -0.02641695,
        0.0524293 ,  0.00377552, -0.04796432,  0.01301267, -0.04575671,
        0.04197851, -0.02661929,  0.03450936,  0.00012681, -0.02314933,
       -0.02818171,  0.04163349, -0.0189718 , -0.02811752, -0.03248736,
       -0.03707757, -0.05186043, -0.02768127,  0.00249372,  0.03107219,
        0.01973241, -0.01251974, -0.04933493, -0.00590042,  0.0518308 ,
       -0.03258428,  0.02098152,  0.06091788, -0.01762014,  0.05057498,
       -0.02464549, -0.00604687, -0.02637284, -0.0593655 , -0.02

In [ ]:
f = open("vec.txt", "w", encoding="utf-8")
f.write("{} {}\n".format(3000, 128))
for i in range(1, 3001):
    # [1list]
    pre = infer.predict([[i]])[0]
    v = " ".join(map(str, pre))
    f.write("{} {}\n".format(tok.index_word[i], v))
f.close()

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format("vec.txt")

In [ ]:
w2v.wv.vocab

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{'the': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e6a0>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e518>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e550>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e6d8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e898>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e908>,
 'br': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1eac8>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1eb00>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e940>,
 'i': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1ea20>,
 'this': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1ea58>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1ea90>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e978>,
 'as': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e9b0>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1e8d0>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7fe8bfb1eb38>,
 'movie': <gensi

In [ ]:
# 詞彙列表: w2v.wv.vocab
w2v.most_similar("create")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cliché', 0.9210569262504578),
 ('sat', 0.9198395013809204),
 ('absurd', 0.9196394681930542),
 ('costs', 0.919398307800293),
 ('embarrassing', 0.9190168976783752),
 ('redeeming', 0.9187518358230591),
 ('hoping', 0.9186166524887085),
 ('garbage', 0.9183362722396851),
 ('fails', 0.9182289838790894),
 ('watchable', 0.9182060956954956)]